# Let's Engineer the Features!

I want to build the model where on a given day, you can review the last rows of the df with the most recent information, and then use that to predict tomorrow's Total AvgTone.

In [ ]:
# Function to aggregate data by date
def aggregate_data(df):
    df_copy = df.copy()
    df_copy['SQLDATE'] = pd.to_datetime(df_copy['SQLDATE'], format='%Y%m%d')
    aggregated = df_copy.groupby('SQLDATE').agg({'EventCode': 'size', 'AvgTone': 'sum'}).reset_index()
    aggregated.columns = ['SQLDATE', 'Number of Events', 'Total AvgTone']
    return aggregated.set_index('SQLDATE')

# Aggregate and reindex data
data_aggregated = aggregate_data(data)
label_data_aggregated = aggregate_data(label_data)

# Create a date range and reindex both dataframes
all_dates = pd.date_range(start=data_aggregated.index.min(), end=pd.Timestamp.today())
reindex_data = data_aggregated.reindex(all_dates, fill_value=0).reset_index()
reindex_label_data = label_data_aggregated.reindex(all_dates, fill_value=0).reset_index()

# Rename columns
reindex_data.columns = ['SQLDATE', 'Number of Events', 'Total AvgTone']
reindex_label_data.columns = ['SQLDATE', 'Number of Events', 'Total AvgTone']

# Display the resulting DataFrames
reindex_data, reindex_label_data


(        SQLDATE  Number of Events  Total AvgTone
 0    2015-01-08                 1       2.007772
 1    2015-01-09                 0       0.000000
 2    2015-01-10                 0       0.000000
 3    2015-01-11                 0       0.000000
 4    2015-01-12                 0       0.000000
 ...         ...               ...            ...
 3636 2024-12-22                 0       0.000000
 3637 2024-12-23                 0       0.000000
 3638 2024-12-24                 0       0.000000
 3639 2024-12-25                 0       0.000000
 3640 2024-12-26                 0       0.000000
 
 [3641 rows x 3 columns],
         SQLDATE  Number of Events  Total AvgTone
 0    2015-01-08                 0            0.0
 1    2015-01-09                 0            0.0
 2    2015-01-10                 0            0.0
 3    2015-01-11                 0            0.0
 4    2015-01-12                 0            0.0
 ...         ...               ...            ...
 3636 2024-12-22      

---

## Saving Test & Train Data

The reindex_data is the propper formulation for the labels of our data set. Let's quickly save test and train sets. I want to capture a really bad protest, and then see if it can predict some of the events in 2023.

Splitting on 2021 seems like a solid choice to capture both types of events.

---

In [ ]:
# Split the data into before and after 2021
train_data_labels = reindex_data[reindex_data['SQLDATE'] < '2021-01-01']
test_data_labels = reindex_data[reindex_data['SQLDATE'] >= '2021-01-01']

# Display the resulting dataframes
print("Train Data:")
display(train_data_labels)
print("\nTest Data:")
display(test_data_labels)

NameError: name 'reindex_data' is not defined

In [ ]:
# Remove the "Number of Events" column
train_data_labels = train_data_labels.drop(columns=['Number of Events'])
test_data_labels = test_data_labels.drop(columns=['Number of Events'])
reindex_data = reindex_data.drop(columns=['Number of Events'])

# Save the train and test data to CSV files
train_data_labels.to_csv('train_data_labels.csv', index=False)
test_data_labels.to_csv('test_data_labels.csv', index=False)
reindex_data.to_csv('data_labels.csv', index=False)